In [1]:
from datasets import load_dataset

dataset = load_dataset("ccdv/arxiv-summarization")
dataset

No config specified, defaulting to: arxiv-summarization/section
Found cached dataset arxiv-summarization (C:/Users/Justin Du/.cache/huggingface/datasets/ccdv___arxiv-summarization/section/1.0.0/fa2c9abf4312afb8660ef8e041d576b8e3943ea96ae771bd3cd091b5798e7cc3)


  0%|          | 0/3 [00:00<?, ?it/s]

DatasetDict({
    train: Dataset({
        features: ['article', 'abstract'],
        num_rows: 203037
    })
    validation: Dataset({
        features: ['article', 'abstract'],
        num_rows: 6436
    })
    test: Dataset({
        features: ['article', 'abstract'],
        num_rows: 6440
    })
})

In [2]:
data_train = dataset['train']
data_test = dataset['test']
data_val = dataset['validation']
data_test

Dataset({
    features: ['article', 'abstract'],
    num_rows: 6440
})

In [3]:
#data_test = data_test.shard(num_shards=644, index=0)
data_val = data_val.shard(num_shards=4, index=0)

In [4]:
data_test

Dataset({
    features: ['article', 'abstract'],
    num_rows: 6440
})

In [5]:
from transformers import AutoTokenizer
from transformers import AutoModelForSeq2SeqLM
from transformers import DataCollatorForSeq2Seq

checkpoint = 'facebook/bart-large-cnn'
tokenizer = AutoTokenizer.from_pretrained(checkpoint)
model = AutoModelForSeq2SeqLM.from_pretrained(checkpoint)
data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

In [6]:
max_length = 1024

def tokenize_function(data):

    model_inputs = tokenizer(
        data["article"],
        truncation=True,
        padding='max_length',
        max_length=max_length,
        return_tensors='pt'
    )

    labels = tokenizer(
        data['abstract'],
        truncation=True,
        padding='max_length',
        max_length=max_length,
        return_tensors='pt'
    )
    model_inputs["labels"] = labels["input_ids"]
    model_inputs["labels_mask"] = labels["attention_mask"]

    return model_inputs

In [7]:
tok = data_test.map(tokenize_function, batched=True)
tok_val = data_val.map(tokenize_function, batched=True)
tok

Loading cached processed dataset at C:/Users/Justin Du/.cache/huggingface/datasets/ccdv___arxiv-summarization/section/1.0.0/fa2c9abf4312afb8660ef8e041d576b8e3943ea96ae771bd3cd091b5798e7cc3\cache-9f1abeaea8a4cdd6.arrow
Loading cached processed dataset at C:/Users/Justin Du/.cache/huggingface/datasets/ccdv___arxiv-summarization/section/1.0.0/fa2c9abf4312afb8660ef8e041d576b8e3943ea96ae771bd3cd091b5798e7cc3\cache-229f7b265240cc62.arrow


Dataset({
    features: ['article', 'abstract', 'input_ids', 'attention_mask', 'labels', 'labels_mask'],
    num_rows: 6440
})

In [8]:
tok = tok.remove_columns(
    data_test.column_names
)

tok_val = tok_val.remove_columns(
    data_val.column_names
)

In [9]:
import evaluate
import nltk


nltk.download("punkt")
rouge_score = evaluate.load('rouge')

[nltk_data] Downloading package punkt to C:\Users\Justin
[nltk_data]     Du\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [10]:
from nltk.tokenize import sent_tokenize

def three_sentence_summary(text):
    return "\n".join(sent_tokenize(text)[:3])

In [11]:
print(three_sentence_summary(data_train[2]["article"]))

the transport properties of nonlinear non - equilibrium dynamical systems are far from well - understood@xcite .
consider in particular so - called ratchet systems which are asymmetric periodic potentials where an ensemble of particles experience directed transport@xcite .
the origins of the interest in this lie in considerations about extracting useful work from unbiased noisy fluctuations as seems to happen in biological systems@xcite .


In [12]:
def evaluate_baseline(dataset, metric):
    summaries = [three_sentence_summary(text) for text in dataset["article"]]
    return metric.compute(predictions=summaries, references=dataset["abstract"])

In [13]:
import pandas as pd

score = evaluate_baseline(data_val, rouge_score)


In [14]:
score

{'rouge1': 0.2590856708565549,
 'rouge2': 0.06230188514757135,
 'rougeL': 0.15406806732496217,
 'rougeLsum': 0.22724666063091628}

In [15]:
rouge_names = ["rouge1", "rouge2", "rougeL", "rougeLsum"]
rouge_dict = dict((rn, round(score[rn]* 100, 2)) for rn in rouge_names)
rouge_dict

{'rouge1': 25.91, 'rouge2': 6.23, 'rougeL': 15.41, 'rougeLsum': 22.72}

In [16]:
from huggingface_hub import notebook_login

notebook_login()

In [17]:
from transformers import Seq2SeqTrainingArguments

batch_size = 8
num_train_epochs = 32
logging_steps = len(data_test)
training_args = Seq2SeqTrainingArguments(
    output_dir='arxiv-bart',
    evaluation_strategy="epoch",
    learning_rate=5.6e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    weight_decay=0.01,
    save_total_limit=3,
    num_train_epochs=num_train_epochs,
    predict_with_generate=True,
    logging_steps=logging_steps,
    label_names=['labels'],
    push_to_hub=True,
    resume_from_checkpoint=True,
    save_steps=10,
)

In [18]:
import numpy as np
from nltk.tokenize import sent_tokenize


def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    # Decode generated summaries into text
    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
    # Replace -100 in the labels as we can't decode them
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    # Decode reference summaries into text
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)
    # ROUGE expects a newline after each sentence
    decoded_preds = ["\n".join(sent_tokenize(pred.strip())) for pred in decoded_preds]
    decoded_labels = ["\n".join(sent_tokenize(label.strip())) for label in decoded_labels]
    # Compute ROUGE scores
    result = rouge_score.compute(
        predictions=decoded_preds, references=decoded_labels, use_stemmer=True
    )
    # Extract the median scores
    result = {key: value* 100 for key, value in result.items()}
    return {k: round(v, 4) for k, v in result.items()}

In [19]:
from transformers import Seq2SeqTrainer

trainer = Seq2SeqTrainer(
    model,
    training_args,
    train_dataset=tok,
    eval_dataset=tok_val,
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)

c:\Users\Justin Du\Documents\GitHub\sf2023\arxiv-bart is already a clone of https://huggingface.co/kworts/arxiv-bart. Make sure you pull the latest changes with `repo.git_pull()`.


In [20]:
trainer.train(resume_from_checkpoint=True)

Loading model from arxiv-bart\checkpoint-110.
The following columns in the training set don't have a corresponding argument in `BartForConditionalGeneration.forward` and have been ignored: labels_mask. If labels_mask are not expected by `BartForConditionalGeneration.forward`,  you can safely ignore this message.
c:\Anaconda\envs\sf2023\lib\site-packages\transformers\optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 6440
  Num Epochs = 32
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 25760
  Continuing training from checkpoint, will skip to saved global_step
  Continuing training from epoch 0
  Continuing traini

  0%|          | 0/110 [00:00<?, ?it/s]

  0%|          | 0/25760 [00:00<?, ?it/s]

You're using a BartTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


KeyboardInterrupt: 

In [ ]:
trainer.evaluate()

The following columns in the evaluation set don't have a corresponding argument in `BartForConditionalGeneration.forward` and have been ignored: labels_mask. If labels_mask are not expected by `BartForConditionalGeneration.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 1609
  Batch size = 8


  0%|          | 0/202 [00:00<?, ?it/s]

{'eval_loss': 0.5431961417198181, 'eval_rouge1': 39.3808, 'eval_rouge2': 13.5776, 'eval_rougeL': 22.2275, 'eval_rougeLsum': 35.1901, 'eval_runtime': 29302.2191, 'eval_samples_per_second': 0.055, 'eval_steps_per_second': 0.007, 'epoch': 0.14}


{'eval_loss': 0.5431961417198181,
 'eval_rouge1': 39.3808,
 'eval_rouge2': 13.5776,
 'eval_rougeL': 22.2275,
 'eval_rougeLsum': 35.1901,
 'eval_runtime': 29302.2191,
 'eval_samples_per_second': 0.055,
 'eval_steps_per_second': 0.007,
 'epoch': 0.14}

In [ ]:
trainer.push_to_hub(commit_message="step 110 eval", tags="summarization")

Saving model checkpoint to arxiv-bart
Configuration saved in arxiv-bart\config.json
Model weights saved in arxiv-bart\pytorch_model.bin
tokenizer config file saved in arxiv-bart\tokenizer_config.json
Special tokens file saved in arxiv-bart\special_tokens_map.json


Upload file runs/Dec08_16-05-21_DESKTOP-4P0HGA6/events.out.tfevents.1670544328.DESKTOP-4P0HGA6.6720.0: 100%|##…

remote: Scanning LFS files for validity, may be slow...        
remote: LFS file scan complete.        
To https://huggingface.co/kworts/arxiv-bart
   190e29e..8eeed41  main -> main

remote: LFS file scan complete.        
To https://huggingface.co/kworts/arxiv-bart
   190e29e..8eeed41  main -> main

Dropping the following result as it does not have all the necessary fields:
{'task': {'name': 'Sequence-to-sequence Language Modeling', 'type': 'text2text-generation'}, 'dataset': {'name': 'arxiv-summarization', 'type': 'arxiv-summarization', 'config': 'section', 'split': 'test', 'args': 'section'}}
To https://huggingface.co/kworts/arxiv-bart
   8eeed41..62112d6  main -> main

   8eeed41..62112d6  main -> main



'https://huggingface.co/kworts/arxiv-bart/commit/8eeed41b4fe138a94c5ff7f4beb8ef67b09206bc'

In [ ]:
# samples = tok['test'][:2000]
# samples = {k: v for k, v in samples.items() if k not in ['abstract', 'article']}
# batch = data_collator(samples)

In [ ]:
# chunk = 500
# tok_abs = []

# for i in (data_test[pos:pos + chunk] for pos in range(0, len(data_test), chunk)):
#     tok_abs.append(tokenizer(i['abstract'], truncation=True, padding=True, return_tensors='tf'))


In [ ]:
# tok_art = []
# for i in (data_test[pos:pos + chunk] for pos in range(0, len(data_test), chunk)):
#     tok_art.append(tokenizer(i['article'], truncation=True, ))

In [ ]:
# import torch
# tok = torch.cat(tok_abs, dim=1)


In [ ]:
# test_tok_abs = tokenizer(data_train['article'], truncation=True, padding='max_length')